In [ ]:
import time
import chess.variant
from IPython.display import display, HTML, clear_output
import random

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

def play_game(player1, player2, visual="svg", pause=0.1):
    use_svg = (visual == "svg")
    board = chess.variant.SuicideBoard()
    
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

def human_player(board):
    # display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

def bot_player(board):
    legal_uci_moves = [move.uci() for move in board.legal_moves]

    if (len(legal_uci_moves) == 1):
        return legal_uci_moves[0]

    treeDepth = 3
    maxi = float('-inf')

    for move in legal_uci_moves:
        score = minimax(0, False, move, treeDepth, board)

        if score > maxi:
            maxi = score
            final_uci = move
    return final_uci

In [ ]:
def evaluationFunc(board, move):
    if board.is_capture(chess.Move.from_uci(move)):
        return len(list(board.legal_moves))
    
    return 0

def minimax(curDepth, maxTurn, move, targetDepth, board):
    board1 = board.copy()
    
    if (curDepth == targetDepth):
        return  evaluationFunc(board1, move)

    board1.push_san(move)
    legal_uci_moves=[move.uci() for move in board1.legal_moves]

    if (maxTurn):
        maxi = float('-inf')
        
        for move in legal_uci_moves:
            score = minimax(curDepth+1, False, move, targetDepth, board1)

            if score > maxi:
                maxi = score

        return maxi
    else:
        mini=float('inf')

        for  move in legal_uci_moves:
            score = minimax(curDepth+1, True, move, targetDepth, board1)

            if score < mini:
                mini = score
                # final_uci = move

        return mini

In [ ]:
play_game(human_player, bot_player)